In [ ]:
import os
import numpy as np
import cv2
import time, datetime
import pickle
import psutil

from imutils import paths
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.metrics import recall_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import auc
from sklearn.metrics import classification_report

In [ ]:
import opendatasets as od

In [1]:
#uncomment this cell to download the dataset
#od.download('https://www.kaggle.com/datasets/iarunava/cell-images-for-detecting-malaria')

In [ ]:

class SimpleDatasetLoader:
    # Method: Constructor
    def __init__(self, preprocessors=None):
        """
        :param preprocessors: List of image preprocessors
        """
        self.preprocessors = preprocessors

        if self.preprocessors is None:
            self.preprocessors = []

    # Method: Used to load a list of images for pre-processing
    def load(self, image_paths, verbose=-1):
        """
        :param image_paths: List of image paths
        :param verbose: Parameter for printing information to console
        :return: Tuple of data and labels
        """
        data, labels = [], []

        for i, image_path in enumerate(image_paths):
            image = cv2.imread(image_path)
            label = image_path.split(os.path.sep)[-2]

            if self.preprocessors is not None:
                for p in self.preprocessors:
                    image = p.preprocess(image)

            data.append(image)
            labels.append(label)

            if verbose > 0 and i > 0 and (i+1) % verbose == 0:
                print('[INFO]: Processed {}/{}'.format(i+1, len(image_paths)))

        return (np.array(data), np.array(labels))

In [ ]:
#Class Preprocessror 
class SimplePreprocessor:
    # Method: Constructor
    def __init__(self, width, height, interpolation=cv2.INTER_AREA):
        """
        :param width: Image width
        :param height: Image height
        :param interpolation: Interpolation algorithm
        """
        self.width = width
        self.height = height
        self.interpolation = interpolation

    # Method: Used to resize the image to a fixed size (ignoring the aspect ratio)
    def preprocess(self, image):
        """
        :param image: Image
        :return: Re-sized image
        """
        return cv2.resize(image, (self.width, self.height), interpolation=self.interpolation)

In [ ]:

from __main__ import SimplePreprocessor
from __main__ import SimpleDatasetLoader

In [ ]:
# Get list of image paths

image_paths = list(paths.list_images(r".\cell-images-for-detecting-malaria\cell_images\cell_images"))

# Initialize SimplePreprocessor and SimpleDatasetLoader and load data and labels
print('[INFO]: Images loading....')
sp = SimplePreprocessor(32, 32)
sdl = SimpleDatasetLoader(preprocessors=[sp])
(data, labels) = sdl.load(image_paths, verbose=13779)

In [ ]:
print(data.shape)
print(labels.shape)

In [ ]:
# Reshape
data = data.reshape((data.shape[0], 3072))

In [ ]:
print('Min: {}, Max: {}'.format(data.min(), data.max()))
Max = float(data.max())
data = data / Max

In [ ]:
data, x_test, labels, y_test = train_test_split(data, labels, test_size=0.3, random_state=42)

In [ ]:
le = LabelEncoder()
labels = le.fit_transform(labels)

In [ ]:
model = SVC(C=1, kernel='linear', gamma="auto")

In [ ]:
x_mean = data.mean(axis=0)
print(x_mean)

data_nm = data - x_mean

print('Min: {}, Max: {}'.format(data_nm.min(), data_nm.max()))
print(data_nm.shape)

In [ ]:
start = datetime.datetime.now()
time.sleep(10)

model.fit(data_nm, labels)
print(psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2)

end = datetime.datetime.now()
diff = (end - start)
print(f"SVM Train Time: {diff}")

In [ ]:
start = datetime.datetime.now()
time.sleep(10)

preds = model.predict(x_test)
print(psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2)

end = datetime.datetime.now()
diff = (end - start)
print(f"SVM Inference Time: {diff}")

In [ ]:
y_test = le.fit_transform(y_test)

In [ ]:
print(classification_report(y_test, preds))
svm_accuracy = metrics.accuracy_score(y_test, preds)
svm_auc = metrics.roc_auc_score(y_test, preds)
svm_mcc = metrics.matthews_corrcoef(y_test, preds)
svm_spec =recall_score(y_test, preds,pos_label=0)
print("Accuracy score: {}".format(svm_accuracy))
print("auc: {}".format(svm_auc))
print("mcc: {}".format(svm_mcc))
print("Specificity: {}".format(svm_spec))

In [ ]:
pickle.dump(model, open("svm.pkl", "wb"))